In [1]:
import json
from datetime import datetime
import time
import xml.etree.ElementTree as ET
import random
from threading import Thread, Lock, local

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Funzione per la conversione delle coordinate
from pyproj import Proj, transform

def trans_coord(coord_X,coord_Y):
    inProj  = Proj("+init=EPSG:4326")
    outProj = Proj("+init=EPSG:32633")
    x,y = transform(inProj,outProj,coord_X,coord_Y)
    return [x,y]

In [4]:
#Generazione orario casuale tra due orari limite
def randomTime(start, end):
    frmt = '%H:%M:%S'

    stime = time.mktime(time.strptime(start, frmt))
    etime = time.mktime(time.strptime(end, frmt))

    ptime = stime + random.random() * (etime - stime)
    dt = datetime.fromtimestamp(time.mktime(time.localtime(ptime)))
    return str(dt.hour).zfill(2)+':'+str(dt.minute).zfill(2)+':'+str(dt.second).zfill(2)

In [5]:
with open('./json_posti_lavoro.json') as json_file:
    posti_lavoro = json.load(json_file)

In [6]:
with open('./json_scuola.json') as json_file:
    scuole = json.load(json_file)

In [7]:
with open('./json_ristoro.json') as json_file:
    ristori = json.load(json_file)

<h1>Agenti Attivi</h1>

In [8]:
with open('./json_agenti_res.json') as json_file:
    agenti = json.load(json_file)

In [9]:
#Definisco possibili routine
routine_lavoratore=['CLMD','CLPMD','CLBLPMD','CLERD','CLEARD','CLMAD']
routine_studente=['CSD','CSPTD','CSERD','CSEARD','CSTAD']

In [10]:
def spostamento_agente(plan,a):
    leg=ET.SubElement(plan,'leg')
    if a['automunito']:
        leg.attrib['mode']='car'
    else:
        leg.attrib['mode']=random.choice(['walk', 'bike'])

In [11]:
def att_random(att_disp,att):
    random.shuffle(att_disp)
    for ad in att_disp:
        if ad['capienza_oraria'][att] >0:
            ad['capienza_oraria'][att]=ad['capienza_oraria'][att]-1
            return ad
    return null

In [12]:
def assegna_attivita(l,plan,a):
    if l=='C': #Casa al mattino
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='casa'
        coord=trans_coord(a['X_residenza'],a['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("06:45:00", "08:15:00")
        spostamento_agente(plan,a)
        
    if l=='D': #Casa alla sera
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='casa'
        coord=trans_coord(a['X_residenza'],a['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        
    if l=='E': #Casa al pomeriggio
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='casa_breve'
        coord=trans_coord(a['X_residenza'],a['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("16:00:00", "18:50:00")
        spostamento_agente(plan,a)
        
    if l=='L': #Lavoro al mattino
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='lavoro_mattino'
        id_lav=a['posto_lavoro']
        p_lavoro=list(filter(lambda pl: pl['id'] == id_lav,posti_lavoro))
        coord=trans_coord(p_lavoro[0]['coord_X'],p_lavoro[0]['coord_Y'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("11:50:00", "13:30:00")
        spostamento_agente(plan,a)
        
    if l=='M': #Lavoro al pomeriggio
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='lavoro_pomeriggio'
        id_lav=a['posto_lavoro']
        p_lavoro=list(filter(lambda pl: pl['id'] == id_lav,posti_lavoro))
        coord=trans_coord(p_lavoro[0]['coord_X'],p_lavoro[0]['coord_Y'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("16:15:00", "17:30:00")
        spostamento_agente(plan,a)
    
    if l=='N': #Lavoro alla sera
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='lavoro_sera'
        id_lav=a['posto_lavoro']
        p_lavoro=list(filter(lambda pl: pl['id'] == id_lav,posti_lavoro))
        coord=trans_coord(p_lavoro[0]['coord_X'],p_lavoro[0]['coord_Y'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("21:15:00", "22:30:00")
        spostamento_agente(plan,a)
        
    if l=='S': #Studio la mattina
        activity=ET.SubElement(plan,'activity')
        if a['tipo']=='attivo':
            activity.attrib['type']='universita_mattino'
        else:
            activity.attrib['type']='scuola_mattino'
        id_scuola=a['scuola_appartenenza']
        scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
        coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("12:10:00", "14:10:00")
        spostamento_agente(plan,a)
        
    if l=='T': #Studio al pomeriggio
        activity=ET.SubElement(plan,'activity')
        if a['tipo']=='attivo':
            activity.attrib['type']='universita_pomeriggio'
        else:
            activity.attrib['type']='scuola_pomeriggio'
            
        id_scuola=a['scuola_appartenenza']
        scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
        coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("16:10:00", "18:10:00")
        spostamento_agente(plan,a)
        
    if l=='P': #Pausa pranzo
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='pausa_pranzo'
        if 'posto_lavoro' in a:
            sez=a['posto_lavoro']
        else:
            sez=a['scuola_appartenenza']
            
        if a['tipo']=='attiva': #Suddivido bambini con ragazzi/adulti
            try: #Cerco un ristorante nelle vicinanze
                rist=list(filter(lambda r: r['sezione_censo']<sez+25 and r['sezione_censo']>sez-25 ,ristori))
                ad=att_random(rist,4)
                coord=trans_coord(ad['coord_X'],ad['coord_Y'])
                leg=ET.SubElement(plan,'leg')
                leg.attrib['mode']='walk'
            except: #Se non lo trova, torna a casa a mangiare
                coord=trans_coord(a['X_residenza'],a['Y_residenza'])
                spostamento_agente(plan,a)
        else:
            id_scuola=a['scuola_appartenenza']
            scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
            coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
            spostamento_agente(plan,a)
                        
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("13:30:00", "14:10:00")
        
    if l=='B': #Bar
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='bar'
        if 'posto_lavoro' in a: #Lavoratore
            sez=a['posto_lavoro']
            try: #Cerco un bar nelle vicinanze del lavoro
                bar=list(filter(lambda r: r['sezione_censo']<sez+25 and r['sezione_censo']>sez-25 and r['tipo_edificio']=='bar',ristori))
                ad=att_random(bar,3)
                coord=trans_coord(ad['coord_X'],ad['coord_Y'])
                leg=ET.SubElement(plan,'leg')
                leg.attrib['mode']='walk'
                
            except: #Se non lo trova, pausa in loco
                id_lav=a['posto_lavoro']
                p_lavoro=list(filter(lambda pl: pl['id'] == id_lav,posti_lavoro))
                coord=trans_coord(p_lavoro[0]['coord_X'],p_lavoro[0]['coord_Y'])
                spostamento_agente(plan,a)
        else:
            if a['tipo'] == 'attiva': #Controllo se studente delle superiori/università
                #Se delle superiori
                sez=a['scuola_appartenenza']
                try: #Cerco un bar nelle vicinanze 
                    bar=list(filter(lambda r: r['sezione_censo']<sez+25 and r['sezione_censo']>sez-25 and r['tipo_edificio']=='bar',ristori))
                    ad=att_random(bar,3)
                    coord=trans_coord(ad['coord_X'],ad['coord_Y'])
                    leg=ET.SubElement(plan,'leg')
                    leg.attrib['mode']='walk'
                except: #Se non lo trova, pausa in loco
                    id_scuola=a['scuola_appartenenza']
                    scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
                    coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
                    spostamento_agente(plan,a)
            else:
                #Altrimenti è un bambino, quindi rimane a scuola
                id_scuola=a['scuola_appartenenza']
                scuola=list(filter(lambda s: s['id'] == id_scuola,scuole))
                coord=trans_coord(scuola[0]['coord_X'],scuola[0]['coord_Y'])
                spostamento_agente(plan,a)
                
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("11:00:00", "11:30:00")
        
    if l=='A': #Aperitivo
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='aperitivo'
        if 'posto_lavoro' in a: #Lavoratore
            sez=a['posto_lavoro']
            try: #Cerco un bar nelle vicinanze del lavoro
                bar=list(filter(lambda r: r['sezione_censo']<sez+10 and r['sezione_censo']>sez-10 and r['tipo_edificio']=='bar',ristori))
                ad=att_random(bar,7)
                coord=trans_coord(ad['coord_X'],ad['coord_Y'])
            except: #Se non lo trova nelle vicinanze, scelgo un bar a caso
                bar=list(filter(lambda r: r['tipo_edificio']=='bar',ristori))
                ad=att_random(bar,7)
                coord=trans_coord(ad['coord_X'],ad['coord_Y'])
        else:
            if a['tipo'] == 'attiva': #Controllo se ragazzo >15 anni
                sez=a['scuola_appartenenza']
                try: #Cerco un bar nelle vicinanze 
                    bar=list(filter(lambda r: r['sezione_censo']<sez+10 and r['sezione_censo']>sez-10 and r['tipo_edificio']=='bar',ristori))
                    ad=att_random(bar,7)
                    coord=trans_coord(ad['coord_X'],ad['coord_Y'])
                except: #Se non lo trova nelle vicinanza, scelgo un bar a caso
                    bar=list(filter(lambda r: r['tipo_edificio']=='bar',ristori))
                    ad=att_random(bar,7)
                    coord=trans_coord(ad['coord_X'],ad['coord_Y'])
            else: #Se bambino, aperitivo diventa merenda a casa
                coord=trans_coord(a['X_residenza'],a['Y_residenza'])
        
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("19:10:00", "20:45:00")
        spostamento_agente(plan,a)
        
    if l=='R': #Cena al ristorante
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='ristorante'
        
        if a['tipo']=='attiva': #Suddivido bambini con ragazzi/adulti
            rist=list(filter(lambda r: r['tipo_edificio']=='restaurant' or r['tipo_edificio']=='fast_food',ristori))
            ad=att_random(rist,8)
            coord=trans_coord(ad['coord_X'],ad['coord_Y'])
        else: #Se bambino, mangia a casa
            coord=trans_coord(a['X_residenza'],a['Y_residenza'])
                        
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=randomTime("19:50:00", "21:15:00")
        spostamento_agente(plan,a)
    

In [13]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

root=ET.Element('population')

#for a in agenti:
for i in range(0,50):
    a=agenti[i]
    
    if 'posto_lavoro' in a:
        routine = routine_lavoratore[random.randint(0, len(routine_lavoratore)-1)]
    else:
        routine = routine_studente[random.randint(0, len(routine_studente)-1)]
        
    person = ET.SubElement(root, 'person')
    person.attrib['id'] = str(a['id'])
    plan=ET.SubElement(person,'plan')

    for r in routine:
        assegna_attivita(r,plan,a)
        
print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-16 11:49:10
Fine = 2021-03-16 11:49:25


________________________

<h1>Agenti non attivi</h1>

In [14]:
#importo json agenti non attivi
with open('./json_agenti_non_attivi_res.json') as json_file:
    agenti_non_attivi = json.load(json_file)

In [15]:
with open('./json_turismo.json') as json_file:
    turismo = json.load(json_file)

In [16]:
with open('./json_spese.json') as json_file:
    spese = json.load(json_file)

In [17]:
#Genero routines orarie non attivi
routines={}
for an in agenti_non_attivi:
    r=[]
    r.append(randomTime("08:30:00","09:45:00")) #uscita di casa
    r.append(randomTime("09:45:01","10:45:00")) #Attività mattina 1
    r.append(randomTime("10:45:01","11:30:00")) #Attività mattina 2
    r.append(randomTime("11:30:01","13:15:00")) #Attività mattina 3
    r.append(randomTime("13:15:01","15:00:00")) #Fine pranzo
    r.append(randomTime("15:00:01","16:30:00")) #Attività pomeriggio 1
    r.append(randomTime("16:30:01","17:45:00")) #Attività pomeriggio 2
    r.append(randomTime("17:45:01","19:30:00")) #Attività pomeriggio 3
    r.append(randomTime("19:30:01","22:00:00")) #Fine cena
    r.append(randomTime("22:00:01","23:30:00")) #Attività serale
    routines[an['id']]=r
    

In [18]:
def assegna_attivita_non_attivi(att,orario,plan,na):
    if att==0: #Casa al mattino
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='casa'
        coord=trans_coord(na['X_residenza'],na['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=orario
        spostamento_agente(plan,na)
        
    if att in [1,2,3,5,6,7]: #Attività mattina 1/2/3 e pomeriggio 5/6/7
        activity=ET.SubElement(plan,'activity')
        #Scelgo casualmente un tipo di attività
        tipo_att=random.choice(['clothes','pharmacy','supermarket','tailor','tobacco','artwork','attraction','museum','park','monument','bar','pub'])
        #Estraggo lista attività disponibili
        if tipo_att in ['clothes','pharmacy','supermarket','tailor','tobacco']:
            activity.attrib['type']="shop"
            att_disp=list(filter(lambda ad: ad['tipo_edificio']==tipo_att,spese))
        if tipo_att in ['artwork','attraction','museum','park','monument']:
            activity.attrib['type']='turismo'
            att_disp=list(filter(lambda ad: ad['tipo_edificio']==tipo_att,turismo))
        if tipo_att in ['bar','pub']:
            activity.attrib['type']='bar'
            att_disp=list(filter(lambda ad: ad['tipo_edificio']==tipo_att,ristori))
        #Prendo casualmente un'attività
        ad=att_random(att_disp,att)
        coord=trans_coord(ad['coord_X'],ad['coord_Y'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=orario
        spostamento_agente(plan,na)
        
    if att==4: #Pranzo
        activity=ET.SubElement(plan,'activity')
        #Scelgo casualmente se mangiare in casa o fuori
        tipo_att=random.choice(['casa','fuori'])
        if tipo_att =="casa": #In casa
            activity.attrib['type']='casa_breve'
            coord=trans_coord(na['X_residenza'],na['Y_residenza'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
        if tipo_att =="fuori": #Fuori
            activity.attrib['type']='pranzo'
            #Individuo attività disponibili
            att_disp=list(filter(lambda ad: ad['tipo_edificio'] in ['restaurant','fast_food'],ristori))
            #Ne scelgo una casualmente
            ad=att_random(att_disp,att)
            coord=trans_coord(ad['coord_X'],ad['coord_Y'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=orario
        spostamento_agente(plan,na)
        
    if att==8: #Cena
        activity=ET.SubElement(plan,'activity')
        #Scelgo casualmente se mangiare in casa o fuori
        tipo_att=random.choice(['casa','fuori'])
        if tipo_att =="casa": #Casa
            activity.attrib['type']='casa_breve'
            coord=trans_coord(na['X_residenza'],na['Y_residenza'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
        if tipo_att =="fuori": #Fuori
            activity.attrib['type']='ristorante'
            #Individuo attività disponibili
            att_disp=list(filter(lambda ad: ad['tipo_edificio'] in ['restaurant','fast_food'],ristori))
            #Ne scelgo una casualmente
            ad=att_random(att_disp,att)
            coord=trans_coord(ad['coord_X'],ad['coord_Y'])
            activity.attrib['x']=str(coord[0])
            activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=orario
        spostamento_agente(plan,na)
        
    if att==9: #Attività serale
        activity=ET.SubElement(plan,'activity')
        #Scelgo casualmente un tipo di attività serale
        tipo_att=random.choice(['attraction','park','monument','bar','pub'])
        #Preparo liste attività disponibili
        if tipo_att in ['attraction','museum','park','monument']:
            activity.attrib['type']='turismo'
            att_disp=list(filter(lambda ad: ad['tipo_edificio']==tipo_att,turismo))
        if tipo_att in ['bar','pub']:
            activity.attrib['type']='serata'
            att_disp=list(filter(lambda ad: ad['tipo_edificio']==tipo_att,ristori))
        #Ne scelgo una casualmente
        ad=att_random(att_disp,att)
        coord=trans_coord(ad['coord_X'],ad['coord_Y'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        activity.attrib['end_time']=orario
        spostamento_agente(plan,na)
        
    if att==10: #Casa alla sera
        activity=ET.SubElement(plan,'activity')
        activity.attrib['type']='casa'
        coord=trans_coord(na['X_residenza'],na['Y_residenza'])
        activity.attrib['x']=str(coord[0])
        activity.attrib['y']=str(coord[1])
        

In [19]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

#root=ET.Element('plans')

#for na in agenti_non_attivi:
for i in range(0,50):
    na=agenti_non_attivi[i]
    
    rout=routines.get(na['id'])
        
    person = ET.SubElement(root, 'person')
    person.attrib['id'] = str(na['id'])
    plan=ET.SubElement(person,'plan')

    for j in range(0,10):
        assegna_attivita_non_attivi(j,rout[j],plan,na)
    assegna_attivita_non_attivi(10,"",plan,na)
        
print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-16 11:49:47
Fine = 2021-03-16 11:50:18


_________________

In [20]:
#Salvo population.xml
with open('population.xml', 'wb') as f:
    f.write('<?xml version="1.0"?>\n<!DOCTYPE population SYSTEM "http://www.matsim.org/files/dtd/population_v6.dtd">'.encode('utf8'))
    ET.ElementTree(root).write(f, 'utf-8')

________________

In [ ]:
rout=routines.get(80965)
rout

In [ ]:
tipo_att=random.choice(['clothes','pharmacy','supermarket','tailor','tobacco','artwork','attraction','museum','park','monument','bar','restaurant','pub','fast_food'])
tipo_att
if tipo_att in ['clothes','pharmacy','supermarket','tailor','tobacco']:
    print(tipo_att)

In [ ]:
att_disp=list(filter(lambda ad: ad['tipo_edificio'] in ['restaurant','fast_food'],ristori))
att_disp[2]
